# The Impact of Lower Interest Rates in Brazilian Stock Market

*By Daniel Deutsch, José Lucas Barretto, Kevin kuhl and Lucas Miguel Agrizzi*

In [8]:
import os
import sys
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver

import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

# Data Collection

In this stage, we want to collect the data necessary for our analysis. In our case, we must collect two kinds of data: Indexes of (which kind? why?) and stock information about the indexes.

## Indexes Collection

Indexes are basically .... (explicar o que são indexes e porque eles serão utilizados na nossa análise).

Given that, the group decided to understand the effects of the index rate on the following indexes (because we considered them the most influential in the brazillian market):

| Index Acronym | Index Name                  | URL           | Sector  |
|:------------- |:----------------------------|:--------------|:-------|
| ICON          | Índice de Consumo           | right-aligned | $1600 |
| IEEX          | Índice de Energia Elétrica  | centered      |   $12 |
| IFNC          | Índice Financeiro           | are neat      |    $1 |
| IMAT          | Índice de Materiais Básicos | are neat      |    $1 |
| IMOB          | Índice Imobiliário          | are neat      |    $1 |
| INDX          | Índice Industrial           | are neat      |    $1 |
| UTIL          | Índice Utilidade Pública    | are neat      |    $1 |

In [80]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException



# Defines the indexes array and the initial url
indexes = ["ICON", "IEEX", "IFNC", "IMAT", "IMOB", "INDX", "UTIL"]
url = "http://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/"

# Creates a webdriver
wd = webdriver.Chrome(executable_path=f"./webdriver/{'linux_chromedriver' if sys.platform == 'linux' else 'win32_chromedriver.exe'}")

# For each index, create a dataframe with the daily values
for index in indexes:
    wd.get(url)
    wd.find_element_by_id("onetrust-accept-btn-handler").click()
    for div in wd.find_elements_by_xpath("//div[@class='card'][@data-equalizer-watch]"):
        if index in div.text:
            div.find_element_by_tag_name("a").click()
            wd.find_element_by_id("aa_8A828D295048C0EF0150733B04F514AA").click()
            #select_year = wd.find_element_by_xpath('//*[@id="selectYear"]')
            #select_semester = wd.find_element_by_xpath('//*[@id="semester"]')

            select_year = WebDriverWait(wd, 20).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='selectYear']"))
            )
            select_semester = WebDriverWait(wd, 20).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='semester']"))
            )

            

            for year in select_year.find_elements_by_tag_name('option'):
                for semester in select_semester.find_elements_by_tag_name('option'):
                    
                    year.click()
                    semester.click()

                    table = BeautifulSoup(wd.find_element_by_tag_name("table").get_attribute('innerHTML'), 'html.parser')


TimeoutException: Message: 


In [147]:
index_dict = {}
for index in index_data['Index'].unique():
    mask = index_data['Index'] == index
    columns = index_data[mask]['Código']
    index_dict[index] = columns.to_list()

In [131]:
# gets stock data from yahoo finance
stock_data = yf.download(tickers = index_data['Código'].to_list(), period='max', interval = "1d")

[*********************100%***********************]  136 of 136 completed


# Data Analysis